In [4]:
import openai
from gpt4_config import GPT4_API_KEY

openai.api_key = GPT4_API_KEY
model = 'gpt-4-1106-preview'

In [1]:
import json
import pandas as pd
import time

In [53]:
def generate_sentences():
    # get 20 sentences about mountains
    messages=[
        {"role": "system", 
         "content": """Here you have the checklist for generating sentences with mountain names:
                    Geographical Diversity: Include mountains from different continents and regions.
                    Contextual Variety: Use sentences from various contexts like travel, history, geography, and news.
                    Sentence Complexity: Mix simple and complex sentences, varying in length and structure.
                    Formal and Informal References: Incorporate both official and colloquial names of mountains.
                    Metaphorical Use: Include sentences where mountain names are used metaphorically.
                    Entity Co-occurrence: Use sentences where mountains are mentioned alongside other entities like rivers, cities, etc.
                    Temporal References: Have historical and current references to mountains.
                    Multiple Entities: Occasionally use sentences with multiple mountain names.
                    Negative Samples: Include sentences with mountain-related words but no mountain name.
                    Multilingual and Transliteration Aspects: If applicable, include names in different languages and transliterations.
                    Cultural and Mythological References: Add references from cultural or mythological texts.
                    Varied Data Sources: Gather sentences from diverse sources to ensure different writing styles.
                    JSON format: {'sentences':[s1,s2,...]} without newline characters"""},
        {"role": "assistant", 
         "content": "generate 50 sentences. Return them in json without any other notes or comments on the task"}
    ]
    response = openai.ChatCompletion.create(model=model, response_format={ "type": "json_object" }, messages=messages)
    #print(response)
    #print(f"Tokens used: {response['usage']}")
    sentences = response['choices'][0]['message']['content']
    sentences_dict = json.loads(sentences)
    
    return sentences_dict

In [13]:
#sentences_dict = generate_sentences()

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "{\n    \"sentences\": [\n        \"Mount Everest, the highest peak on Earth, straddles the border between Nepal and China's autonomous region of Tibet.\",\n        \"The hikers embarked on an arduous journey to ascend Mount Kilimanjaro, Africa's tallest mountain, which is also a dormant volcano.\",\n        \"Denali, formerly known as Mount McKinley, is North America's highest mountain peak and is renowned for its stunning beauty and challenging terrain.\",\n        \"The Inca citadel of Machu Picchu is nestled high in the Andes mountains, revealing the architectural ingenuity of ancient civilizations.\",\n        \"In Greek mythology, Mount Olympus was regarded as the abode of the gods, a place where Zeus and his pantheon resided.\",\n        \"Mount Fuji's symmetrical cone, a celebrated symbol of Japan, is a frequent subject in art and literature due to its cultural signific

In [16]:
#print(sentences_dict)

{'sentences': ["Mount Everest, the highest peak on Earth, straddles the border between Nepal and China's autonomous region of Tibet.", "The hikers embarked on an arduous journey to ascend Mount Kilimanjaro, Africa's tallest mountain, which is also a dormant volcano.", "Denali, formerly known as Mount McKinley, is North America's highest mountain peak and is renowned for its stunning beauty and challenging terrain.", 'The Inca citadel of Machu Picchu is nestled high in the Andes mountains, revealing the architectural ingenuity of ancient civilizations.', 'In Greek mythology, Mount Olympus was regarded as the abode of the gods, a place where Zeus and his pantheon resided.', "Mount Fuji's symmetrical cone, a celebrated symbol of Japan, is a frequent subject in art and literature due to its cultural significance.", 'During the climbing season, base camp at Everest becomes a bustling community of mountaineers, guides, and sherpas.', 'The Rocky Mountains stretch across North America, influen

In [54]:
for _ in range(20):
    print(f'{_}: generating 50.')
    sentences_dict = generate_sentences()
    #print(sentences_dict)
    with open('sentences.txt', 'a', encoding="utf-8") as file:
        for sentence in sentences_dict['sentences']:
            file.write(sentence + '\n')
    print(f'{_}: done.')

0: loading first 50.
{'sentences': ['Mount Everest, located in the Himalayas, is the highest peak on Earth.', "Kilimanjaro's snow-capped summit is one of the most iconic sights in Africa.", 'The Rocky Mountains span across North America, from Canada to the southwestern United States.', 'Fujisan, more commonly known as Mount Fuji, is an iconic symbol of Japan.', "The Andes stretch along South America's western edge, hosting diverse ecosystems.", 'Mont Blanc is the highest mountain in the Alps and a popular destination for climbers.', 'Denali, formerly known as Mount McKinley, is the tallest peak in North America.', "Many climbers dream of conquering the Seven Summits, starting with Africa's Kilimanjaro.", 'Trekking in the Andes often includes a visit to Machu Picchu, the ancient Incan city.', 'The Appalachian Mountains are known for their fall foliage, drawing tourists every autumn.', 'The Urals form a natural border between European and Asian Russia.', 'Mount Vesuvius looms over the ci

In [5]:
def tag_sentence(sentence):
    messages=[
        {"role": "system",
         "content": """IOB (Inside, Outside, Beginning) tagging scheme:
                    MNT: For names of mountains.
                    MNTREL: General mountain-related term (e.g., summit, peak, ridge).
                    LOC: For broader geographical locations.
                    COUNTRY/CITY: For countries and cities related to mountains.
                    GEO: For other geographical features like rivers, forests.
                    PARK: For national parks or reserves including mountains.
                    HIST: For historical events related to mountains.
                    CULT: For cultural or religious sites associated with mountains.
                    PER: For names of people associated with mountains.
                    ORG: For organizations related to mountains.
                    DATE: For dates or times relevant to mountain-related events.
                    O: For tokens that don't belong to any specific category.
                    In the IOB scheme, "B-" indicates the beginning of an entity, and "I-" indicates that the token is inside an entity but not the first token.
                    (do not add any other tags!)
                    Required json format:
                    {"sentence_id": 0,
                    "tokens": [
                    {"token": "Mount", "pos": "NNP", "tag": "B-MNT"},
                    {"token": "Everest", "pos": "NNP", "tag": "I-MNT"},...]}"""},
        {"role": "assistant",
         "content": f'separate the next sentence into tokens and tag them with given tags(if applicable), add for each token its part of speech(pos) in abbreviation. Return content in json format. Do not add to the response any other your comments or notes! Sentence: {sentence}'}
    ]
    response = openai.ChatCompletion.create(model=model, response_format={ "type": "json_object" }, messages=messages)
    tagged = (response['choices'][0]['message']['content'])
    return tagged

In [6]:
json_list = []
with open('sentences.txt', 'r') as file:
    for sentence_number, sentence in enumerate(file, start=1):
        if sentence_number % 20 == 0 :
            print('---Short break---')
            time.sleep(60)
        print(f'{sentence_number}: loading...\t')
        tagged =  tag_sentence(sentence.strip())
        json_tagged = json.loads(tagged)
        json_tagged['sentence_id'] = sentence_number
        json_list.append(json_tagged)
        print(f'{sentence_number}: done.\t')

1: loading...	
1: done.	
2: loading...	
2: done.	
3: loading...	
3: done.	
4: loading...	
4: done.	
5: loading...	
5: done.	
6: loading...	
6: done.	
7: loading...	
7: done.	
8: loading...	
8: done.	
9: loading...	
9: done.	
10: loading...	
10: done.	
11: loading...	
11: done.	
12: loading...	
12: done.	
13: loading...	
13: done.	
14: loading...	
14: done.	
15: loading...	
15: done.	
16: loading...	
16: done.	
17: loading...	
17: done.	
18: loading...	
18: done.	
19: loading...	
19: done.	
---Short break---
20: loading...	
20: done.	
21: loading...	
21: done.	
22: loading...	
22: done.	
23: loading...	
23: done.	
24: loading...	
24: done.	
25: loading...	
25: done.	
26: loading...	
26: done.	
27: loading...	


ServiceUnavailableError: The server is overloaded or not ready yet.

In [21]:
with open('sentences.txt', 'r', encoding='utf-8') as file:
    for sentence_number, sentence in enumerate(file, start=1):
        if sentence_number < 91:
            continue
        if sentence_number % 20 == 0 :
            print('---Short break---')
            time.sleep(60)
        print(f'{sentence_number}: loading...\t')
        tagged =  tag_sentence(sentence.strip())
        json_tagged = json.loads(tagged)
        json_tagged['sentence_id'] = sentence_number
        json_list.append(json_tagged)
        print(f'{sentence_number}: done.\t')

91: loading...	
91: done.	
92: loading...	
92: done.	
93: loading...	
93: done.	
94: loading...	
94: done.	
95: loading...	
95: done.	
96: loading...	
96: done.	
97: loading...	
97: done.	
98: loading...	
98: done.	
99: loading...	
99: done.	
---Short break---
100: loading...	
100: done.	
101: loading...	
101: done.	
102: loading...	
102: done.	
103: loading...	
103: done.	
104: loading...	
104: done.	
105: loading...	
105: done.	
106: loading...	
106: done.	
107: loading...	
107: done.	
108: loading...	
108: done.	
109: loading...	
109: done.	
110: loading...	
110: done.	
111: loading...	
111: done.	
112: loading...	
112: done.	
113: loading...	
113: done.	
114: loading...	
114: done.	
115: loading...	
115: done.	
116: loading...	
116: done.	
117: loading...	
117: done.	
118: loading...	
118: done.	
119: loading...	
119: done.	
---Short break---
120: loading...	
120: done.	
121: loading...	
121: done.	
122: loading...	
122: done.	
123: loading...	
123: done.	
124: loading...	
124: don

KeyboardInterrupt: 

In [22]:
print(len(json_list))

490


In [23]:
# Write the list to a file as a JSON array
with open('data-tagged.json', 'w') as file:
    json.dump(json_list, file)

In [24]:
with open('data-tagged.json', 'r') as file:
    data = json.load(file)

# Flatten the nested structure
# 'record_path' is the path to the nested list (in this case, 'tokens')
# 'meta' contains additional fields you want to keep (in this case, 'sentence_id')
df = pd.json_normalize(data, record_path='tokens', meta=['sentence_id'])
df = df[['sentence_id', 'token', 'pos', 'tag']] 

In [25]:
df

,sentence_id,token,pos,tag
0,1,Mount,NNP,B-MNT
1,1,Everest,NNP,I-MNT
2,1,is,VBZ,O
3,1,known,VBN,O
4,1,as,IN,O
...,...,...,...,...
8265,490,a,DT,O
8266,490,mountain,NN,B-MNTREL
8267,490,range,NN,I-MNTREL
8268,490,in,IN,O


In [26]:
df.tag.unique()

array(['B-MNT', 'I-MNT', 'O', 'B-MNTREL', 'B-LOC', 'I-LOC', 'B-CITY',
       'I-CITY', 'B-COUNTRY', 'I-COUNTRY', 'B-PARK', 'I-PARK', 'I-MNTREL',
       'B-CULT', 'I-CULT', 'HIST', 'B-GEO', 'I-GEO', 'B-DATE', 'B-HIST',
       'MNTREL', 'I-DATE', 'B-PER', 'I-HIST', 'B-COUNTRY/CITY', 'B-ORG',
       'I-ORG', 'LOC', 'I-PER', 'DATE'], dtype=object)